In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt, seaborn as sn, numpy as np, pandas as pd
sn.set_context('notebook')

# Markov chain Monte Carlo (MCMC) methods

In the previous notebook we saw how Monte Carlo (MC) methods can be used to draw representative samples from difficult distributions and also to integrate functions have have no analytical solution. However, we also saw than many basic MC techniques become inefficient for high-diminsional problems.

There are many ways in which the simple MC algorithms from the previous notebook can be improved, but in this notebook we're going to focus on just one very powerful sub-set of MC methods: **Markov chain Monte Carlo (MCMC)**.

One characteristic of the MC methods discussed so far is that they rely on **independent random sampling**. This independence is a useful property, but it also leads to inefficiencies. Ultimately, we want to draw a representative sample from some complex distribution, $f(x)$ (which might be e.g. our posterior distribution from a model calibration exercise). To do this, we need to draw more samples from regions of $f(x)$ with high probability density and fewer samples from regions with low probability density. Because standard MC approaches use independent sampling, the algorithms do not "know" when they're in high density portions of the parameter space. To improve matters, we might construct an algorithm with a "memory" of its previous samples, which it could use identify relatively high density areas compared to where it's been previously. Having identified such a region, the algorithm might decide to draw more samples from that part of the parameter space, threby producing a representative sample from $f(x)$ more quickly.  

This idea introduces a few new problems. In particular, if we choose the next sample, $x_{i+1}$, based on the current sample, $x_i$, then the samples will be auto-correlated (i.e. not independent), which means that each individual sample contains less new information about $f(x)$ than if the samples were independent. On the other hand, this approach may be more efficient than independent sampling strategies, because the algorithm will focus on the most important regions of the parameter space. We may therefore be able to get away with fewer samples, even if the information of each one is reduced.

## 1. Markov chains

A [Markov chain](https://en.wikipedia.org/wiki/Markov_chain) is a random process where the next state of the chain depends *only* on the current state and not on the sequence of events that preceded it. The most commonly used analogy for a Markov chain is that of a drunken man staggering along a street: his location after his next step is determined by his current location, plus a random perturbation. He therefore takes a **random walk** through the landscape.

At the risk of stretching this analogy too far, the Markov chains that we will use are more like drunken *mountaineers*, because they will also possess a desire or tendency to move *uphill* towards regions of higher probability density. However, it is important to note  that they do not *always* move uphill (sometimes they descend or contour a bit), but in general they like to spend more time on the high ground than down in the valley bottoms. 

Note that if our "mountaineers" *did* always move uphill in our probability landscape we would actually be constructing an *optimiser* (and an inefficient one at that) rather than a sampler. If we just want to find the highest point in our landscape there are plenty of better options out there - see the section on Maximum Likelihood Estimation at the end of notebook 2, for example. However, we don't want an optimiser; we want to draw a representative sample from our target distribution, and this means our "mountaineers" must occasionally go down to sample from the low probability "valleys" as well as spending most of our time on the high probability "mountain tops".

## 2. Brief reminder on rejection sampling

In the last notebook, we discussed **non-uniform rejection sampling**, where we introduced a function, $mQ(x)$ which we know to be greater than $f(x)$ everywhere in our region of interest. We then drew independent samples, $x_i$, from $Q(x)$ and accepted them as samples from $f(x)$ if

$$y_i \leq f(x_i) \qquad where \qquad y_i \sim U(0, mQ(x_i))$$

In other words, where the $y_i$ are drawn from a uniform distribution between 0 and $mQ(x_i)$. 

Before we move one, we will first re-write this acceptance rule:

$$U(0, mQ(x_i)) = mQ(x_i) \times U(0,1)$$

Therefore accept when 

$$y_i \leq \frac{f(x_i)}{mQ(x_i)} \qquad where \qquad y_i \sim U(0,1)$$

This gives exactly the same result as above, but this time we calculate the ratio, $\alpha = \frac{f(x_i)}{mQ(x_i)}$, and we also draw a random number, $y_i$, uniformly from the range between 0 and 1. If $y_i \leq \alpha$ then the point $x_i$ is accepted as a sample from $f(x)$.

## 3. The Metropolis algorithm

The most basic MCMC algorithm is the **[Metropolis algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm)**, which is both remarkably simple and extremely powerful. The algorithm proceeds as follows:

1. Begin at an arbitrary point, $x_0$, within the parameter space. <br><br>

2. Choose an arbitrary distribution, $Q(x)$, for determining the random component of the step. This is called the **jump** or **proposal** distribution and for the Metropolis algorithm it must be **symmetric** - in most cases it is chosen to be **Gaussian**. The jump distribution is centred on the current location, $x_0$ and provides a probability distribution for selecting a candidate location for the next step, $x_1$. If the distribution is Gaussian then points closer to $x_0$ are more likely to be chosen as candidates than those further away.<br><br>

3. Evaluate $\alpha = \frac{f(x_1)}{f(x_0)}$.

    * If $\alpha \geq 1$ then $x_1$ has a higher probability density than $x_0$, so because out "drunken mountaineers" like to move uphill, we accept $x_1$ as the next step in the Markov chain and move to that location.<br><br>
    
    * If $\alpha < 1$, draw a value, $y_i$ from a uniform distribution between 0 and 1. Accept $x_1$ as the next step in the chain if $y_i \leq \alpha$, otherwise reject $x_1$ and remain at $x_0$ for this step. Note that this rejection rule is essentially the same as for **rejection sampling** (see above), except here we're applying it to subsequent steps in a Markov chain. This rule means that, when $\alpha < 1$, the probability of the chain moving "downhill" to an area with lower probability density is $\alpha$. <br><br>
    
4. Draw another candidate point, $x_2$ from the proposal distribution, $Q(x)$. If $x_1$ was accepted then the proposal distribution is now centred on $x_1$; if it was rejected the chain is still at $x_0$. Repeat step 3.

It can be proved that, as the number of steps in the chain becomes large, the distribution of samples in the chain converges on the distribution of $f(x)$. Although I'm not going to reproduce the proof here, I hope you can see intuitively why it works:

* The Markov chain takes a random walk through the probability landscape defined by $f(x)$. The chain has a tendency to move uphill, because when the newly proposed point has higher probability density than the current point the proposal is *always* accepted. <br><br>

* However, the proposed point is also *sometimes* accepted even when its probability density is lower, so the algorithm can still move "downhill". If the probability density at $x_{i+1}$ is only half that at $x_i$, then the probability of accepting $x_{i+1}$ as the next point in the chain is $\frac{f(x_{i+1})}{f(x_i)} = 0.5$. In this way the chain has a tendency to stay in (and return large numbers of samples from) high-density regions of $f(x)$, while only visiting lower density regions in proportion to their relative density. 

* The sequence of samples in the chain therefore converges on the distribution of $f(x)$. Because the chain "knows" when to stay in high density regions and when to move downhill, it can do this more efficiently than the standard MC algorithms considered in the previous notebook.

In [ ]:
# Metropolis algorithm

Stress that guaranteed to converge, but not in reasoanble time - quite an amazing result